In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import torch
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
directories = ['campus']
output_dir = './reduced_frames'

dir_files = {}

for directory in directories:
    for dirname, _, filenames in os.walk('/kaggle/input/multi-camera-person-tracking-frames/frames/' + directory):
        files = []
        filenames = map(lambda x: x.split(".")[0],filenames)
        for filename in sorted(filenames,key=int):
            files.append(os.path.join(dirname, filename + ".jpg"))
            print(os.path.join(dirname, filename + ".jpg"))
        dir_files[directory] = files

In [ ]:
from PIL import Image
## Storing a Dict of all the files with the label of a particular Scene
images = {}
for key in dir_files:
    vecs = []
    for file in dir_files[key]:
        vecs.append(Image.open(file).convert('RGB'))
        print(len(vecs))
    images[key] = vecs
    print(images)

In [ ]:
len(images)

In [ ]:
import imagehash
# Creating ImageHashes for each of these frame to check similarity between them
all_hashes = []
cnt = 0
for img in images['campus']:
    print(imagehash.average_hash(img))
    all_hashes.append(imagehash.average_hash(img))
    print(cnt)
    
    cnt += 1

In [ ]:
# Main Algorithm for reduction which compares our current frame with a series of next frames until 
# a threshold change is detected

def reduce_frames():
    prev_f_idx = 0
    next_f_idx = 1
    init_val = all_hashes[next_f_idx] - all_hashes[prev_f_idx]  
    needed_frames = [prev_f_idx,next_f_idx]
    threshold = 3
    while(next_f_idx < len(all_hashes) - 1):
        next_f_idx += 1
        if(all_hashes[next_f_idx] - all_hashes[prev_f_idx] >= threshold):
            needed_frames.append(next_f_idx)
            prev_f_idx = next_f_idx
        else:
            continue
        print(len(needed_frames))
    return needed_frames

reduced = reduce_frames()
        

In [ ]:
len(reduced)

In [ ]:
# Storing the output of these indexes in a file

with open("reduced_frames_" + list(images.keys())[0] + ".txt", "w") as file1:
    # Writing data to a file
    for num in reduced:
        print(str(num) + "-")
        file1.write(str(num) + "-")

file1.close()

In [ ]:
## Displaying the top frames for a scene
import matplotlib.pyplot as plt

w = 5
h = 5

load_img = lambda filename: np.array(Image.open(f"{filename}").resize((200, 200)))

_, axes_list = plt.subplots(h, w, figsize=(12, 12)) # define a grid of (w, h)

# print(axes_list)

cnt = 0
imgs_to_copy = []
for axes in axes_list:
    for ax in axes:
        ax.axis('off')
        img = "/kaggle/input/multi-camera-person-tracking-frames/frames/"+ list(images.keys())[0] + "/" + str(reduced[cnt]) + ".jpg"
        imgs_to_copy.append(img)
        cnt += 1
        ax.imshow(load_img(img)) # load and show
#         ax.set_title(img)

In [ ]:
# Copying the files from our input to a reduced_frames folder
!rm -rf reduced_frames
!mkdir reduced_frames
# !cp 
import shutil

for cnt in range(len(reduced)):
    img = "/kaggle/input/multi-camera-person-tracking-frames/frames/"+ list(images.keys())[0] + "/" + str(reduced[cnt]) + ".jpg"
    shutil.copyfile(img, output_dir + "/" +  str(reduced[cnt]) + ".jpg")


In [ ]:
!ls -l ./reduced_frames
# !zip ./reduced_frames 

In [ ]:
!tar -czvf red_frames_new.tar.gz ./reduced_frames

In [ ]:
from IPython.display import FileLink

FileLink(r'red_frames_new.tar.gz')
# FileLink(r'reduced_frames_' + list(images.keys())[0] + ".txt")